# Using MobileNets to build a Dog Breed Classifier

### Derived from: https://github.com/ferhat00/Deep-Learning/tree/master/Transfer%20Learning%20CNN

In [1]:
import tensorflow as tf
from tensorflow import keras

print(tf.VERSION)
print(keras.__version__)

1.12.0
2.1.6-tf


In [2]:
import numpy as np

dog_names = np.load('./labels/dog_names.npy')

In [4]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
# train_targets are one-hot encodingss
train_files, train_targets = load_dataset('dog-images-training/train')
valid_files, valid_targets = load_dataset('dog-images-training/valid')
test_files, test_targets = load_dataset('dog-images-training//test')

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files)) 

Using TensorFlow backend.


There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


## Preprocess Input

In [3]:
from keras.applications import mobilenet
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications import imagenet_utils
from tqdm import tqdm

Using TensorFlow backend.


In [12]:
from keras.preprocessing import image

def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded_dims)


## Predict with MobileNet

In [5]:
mobile_nets_model = MobileNet()

In [8]:
from keras.preprocessing import image
from IPython.display import Image

filename = './dogImages/Affenpinscher'
preprocessed_image = prepare_image(filename)
predictions = mobile_nets_model.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

[[('n02110627', 'affenpinscher', 0.85866743),
  ('n02106382', 'Bouvier_des_Flandres', 0.058416784),
  ('n02105251', 'briard', 0.030900812),
  ('n02112706', 'Brabancon_griffon', 0.010544327),
  ('n02097130', 'giant_schnauzer', 0.010492797)]]

In [7]:
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D

#imports the mobilenet model and discards the last 1000 neuron layer.
# Important: Make sure to add input_shape otherwise the tflite converter will complain about `None` 
base_model = MobileNet(weights='imagenet', include_top = False, input_shape=(224, 224, 3)) 

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(1024, activation='relu')(x) #dense layer 2
x = Dense(512, activation='relu')(x) #dense layer 3
preds = Dense(2, activation='softmax')(x) #final layer with softmax activation

final_model = Model(inputs=base_model.input, outputs=preds)
final_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

## Adding training dataset

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.optimizers import Adam

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('./dogs/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)



## note: switch using 2 classes for faster processing; change to 133 classes later

Found 128 images belonging to 2 classes.


In [9]:
final_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
step_size_train=train_generator.n//train_generator.batch_size
final_model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

Epoch 1/5
4/4 [==============================] - 29s 7s/step - loss: 0.4089 - acc: 0.8203
Epoch 2/5
4/4 [==============================] - 19s 5s/step - loss: 0.0017 - acc: 1.0000
Epoch 3/5
4/4 [==============================] - 17s 4s/step - loss: 9.5067e-04 - acc: 1.0000
Epoch 4/5
4/4 [==============================] - 17s 4s/step - loss: 1.2666e-07 - acc: 1.0000
Epoch 5/5
4/4 [==============================] - 17s 4s/step - loss: 1.3458e-07 - acc: 1.0000


## Test

In [20]:
final_model.save('keras_models/weights.single_model_mobilenet.hdf5')

In [19]:
filename = './dogImages/Bloodhound'
preprocessed_image = prepare_image(filename)
predictions = final_model.predict(preprocessed_image)
np.argmax(predictions)

1